## Mounting the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the essential libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import tensorflow as tf
tf.__version__
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## Importing the dataset

In [3]:
dataset_train = pd.read_csv('drive/MyDrive/emails.csv',nrows=10000)
dataset_train.duplicated().sum()

0

In [4]:
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }

def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [5]:
email_df = pd.DataFrame(parse_into_emails(dataset_train.message))
email_df.drop(email_df.query("body == '' | to == '' | from_ == ''").index, inplace=True)

In [6]:
email_df.head()

,body,to,from_
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com


## Getting the shape of the dataset

In [7]:
email_df.shape

(9464, 3)

## Preprocessing the dataset

In [8]:
def preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    '''
    Preprocess a string.
    :parameter
        :param text: string - name of column containing text
        :param lst_stopwords: list - list of stopwords to remove
        :param flg_stemm: bool - whether stemming is to be applied
        :param flg_lemm: bool - whether lemmitisation is to be applied
    :return
        cleaned text
    '''

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
email_df["body_clean"] = email_df["body"].apply(lambda x: preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
email_df.head()

,body,to,from_,body_clean
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,test successful way go
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,randycan send schedule salary level everyone t...
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com,greghow either next tuesday thursdayphillip


In [13]:
email_df.groupby('to').describe()

body         \
                                                   count unique   
to                                                                
"'Arnold, John'" <john.arnold@enron.com>               9      3   
"'Eric Bass'" <Eric.Bass@enron.com>                    1      1   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>          2      2   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>      3      1   
"'K. Bass'" <daphneco64@bigplanet.com>                 1      1   
...                                                  ...    ...   
ywang@enron.com                                        6      2   
ywang@enron.com, patti.sullivan@enron.com, phil...    18      2   
zalaywan@caiso.com                                     3      1   
zalaywan@caiso.com, eschmid@caiso.com, crobinso...     3      1   
zeu3rk@msn.com                                         1      1   

                                                                                                       \
                                                                                                  top   
to                                                                                                      
"'Arnold, John'" <john.arnold@enron.com>            personal business.PMJohn -I completely underst...   
"'Eric Bass'" <Eric.Bass@enron.com>                 I hope you were only thinking good things!I ha...   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>       It only takes about 10 minutes to get to highl...   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>   I will not be able to attend the meeting but d...   
"'K. Bass'" <daphneco64@bigplanet.com>              I think we are going to stay in town and meet ...   
...                                                                                               ...   
ywang@enron.com                                               please add mike grigsby to distribution   
ywang@enron.com, patti.sullivan@enron.com, phil...  jane.m.tholt@enron.com, mike.grigsby@enron.com...   
zalaywan@caiso.com                                  put this into the congestion redesign file if ...   
zalaywan@caiso.com, eschmid@caiso.com, crobinso...  mary.hain@enron.comBelow is a summary of my SW...   
zeu3rk@msn.com                                      Got debt? We can help using Debt Consolidation...   

                                                        from_         \
                                                   freq count unique   
to                                                                     
"'Arnold, John'" <john.arnold@enron.com>              3     9      2   
"'Eric Bass'" <Eric.Bass@enron.com>                   1     1      1   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>         1     2      1   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>     3     3      1   
"'K. Bass'" <daphneco64@bigplanet.com>                1     1      1   
...                                                 ...   ...    ...   
ywang@enron.com                                       3     6      1   
ywang@enron.com, patti.sullivan@enron.com, phil...   15    18      1   
zalaywan@caiso.com                                    3     3      1   
zalaywan@caiso.com, eschmid@caiso.com, crobinso...    3     3      1   
zeu3rk@msn.com                                        1     1      1   

                                                                                            \
                                                                                       top   
to                                                                                           
"'Arnold, John'" <john.arnold@enron.com>                       John.Arnold@enron.com [SMTP   
"'Eric Bass'" <Eric.Bass@enron.com>                                    eric.bass@enron.com   
"'Eric_Bass@enron.com'" <Eric_Bass@enron.com>                          eric.bass@enron.com   
"'John.Arnold@enron.com'" <John.Arnold@enron.com>                    

In [14]:
email_df.groupby('from_').describe()

body         \
                                                   count unique   
from_                                                             
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...     3      1   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...     1      1   
"Adrian Clark" <AClark@firstcallassociates.com>...     1      1   
"Allen, Phillip K." <Phillip.K.Allen@ENRON.com>        1      1   
"Andy Colman" <andy@spectronenergy.com>@ENRON          1      1   
...                                                  ...    ...   
wtashnek@aol.com                                       1      1   
yahoo-delivers@yahoo-inc.com                           8      4   
yevgeny.frolov@enron.com                               1      1   
yild@zdemail.zdlists.com                               3      1   
zachary.streight@enron.com                             1      1   

                                                                                                       \
                                                                                                  top   
from_                                                                                                   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...  Please send it to me.johnJohn,We are trying to...   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...  Please send it to me.johnJohn,We are trying to...   
"Adrian Clark" <AClark@firstcallassociates.com>...  I know a set up when I see one-----Original Me...   
"Allen, Phillip K." <Phillip.K.Allen@ENRON.com>     Ceci,I would like to stay current on listings ...   
"Andy Colman" <andy@spectronenergy.com>@ENRON       not mad at you-----Original Message-----Jon, c...   
...                                                                                               ...   
wtashnek@aol.com                                    I'm just checking to see if you got my e-mail ...   
yahoo-delivers@yahoo-inc.com                        [IMAGE]Yahoo! sent this email to you because y...   
yevgeny.frolov@enron.com                            kirk.mcdaniel@enron.comkirk.mcdaniel@enron.com...   
yild@zdemail.zdlists.com                            as links, visit the Yahoo! Internet Life home ...   
zachary.streight@enron.com                          John,I have attached a link to the first itera...   

                                                           to         \
                                                   freq count unique   
from_                                                                  
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...    3     3      1   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...    1     1      1   
"Adrian Clark" <AClark@firstcallassociates.com>...    1     1      1   
"Allen, Phillip K." <Phillip.K.Allen@ENRON.com>       1     1      1   
"Andy Colman" <andy@spectronenergy.com>@ENRON         1     1      1   
...                                                 ...   ...    ...   
wtashnek@aol.com                                      1     1      1   
yahoo-delivers@yahoo-inc.com                          3     8      2   
yevgeny.frolov@enron.com                              1     1      1   
yild@zdemail.zdlists.com                              3     3      1   
zachary.streight@enron.com                            1     1      1   

                                                                                                       \
                                                                                                  top   
from_                                                                                                   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...                                  JPECHER@ENRON.COM   
"ALLYSON FELLER, BLOOMBERG/ NEW YORK" <AFELLER@...                                  JPECHER@ENRON.COM   
"Adrian Clark" <AClark@firstcallassociates.com>...                                        Maggi, Mike   
"Al

In [15]:
email_df.groupby('body_clean').describe()

body         \
                                                   count unique   
body_clean                                                        
                                                       9      4   
001nevereditthisline002this advertisement sent ...     1      1   
01 attachment free virus scan mail forwarded ph...     4      1   
010101 herald actual millenium unique date repe...     3      1   
072701francishere file need excel thank helpphi...     1      1   
...                                                  ...    ...   
zf trading 4                                           4      1   
zf wow would thunk prompt gas 6 zf aswide last ...     3      1   
zip250mb usb drive take anywhere_______________...     1      1   
zip250mb usb drive take anywhere_______________...     1      1   
zlaziccaisocom rabernathycaisocom pmackincaisoc...     3      1   

                                                                                                       \
                                                                                                  top   
body_clean                                                                                              
                                                                       AM ---------------------------   
001nevereditthisline002this advertisement sent ...  --001nevereditthisline002---------------------...   
01 attachment free virus scan mail forwarded ph...  /01 Attachment is free from viruses. Scan Mail...   
010101 herald actual millenium unique date repe...  01.01.01 heralds the actual millenium. A uniqu...   
072701francishere file need excel thank helpphi...  of 07/27/01Francis,Here are the files we need ...   
...                                                                                               ...   
zf trading 4                                                                            z/f trading 4   
zf wow would thunk prompt gas 6 zf aswide last ...  Z/F !!!!   wow.    Who would have thunk it.  P...   
zip250mb usb drive take anywhere_______________...  ==============================================...   
zip250mb usb drive take anywhere_______________...  ==============================================...   
zlaziccaisocom rabernathycaisocom pmackincaisoc...  zlazic@caiso.com, rabernathy@caiso.com, pmacki...   

                                                           to         \
                                                   freq count unique   
body_clean                                                             
                                                      4     9      4   
001nevereditthisline002this advertisement sent ...    1     1      1   
01 attachment free virus scan mail forwarded ph...    4     4      1   
010101 herald actual millenium unique date repe...    3     3      1   
072701francishere file need excel thank helpphi...    1     1      1   
...                                                 ...   ...    ...   
zf trading 4                                          4     4      1   
zf wow would thunk prompt gas 6 zf aswide last ...    3     3      1   
zip250mb usb drive take anywhere_______________...    1     1      1   
zip250mb usb drive take anywhere_______________...    1     1      1   
zlaziccaisocom rabernathycaisocom pmackincaisoc...    3     3      1   

                                                                                                       \
                                                                                                  top   
body_clean                                                                                              
                                                                              John Arnold/HOU/ECT@ECT   
001nevereditthisline002this advertisement sent ...                              jarnold@ect.enron.com   
01 attachment free virus scan mail forwarded ph...                                Anne.Bike@enron.com   
010

In [16]:
fraud_related_words = ['fraud', 'scam', 'phishing', 'malware', 'deception', 'hoax']  # Add more fraud-related words if needed

email_df['fraud'] = email_df['body_clean'].apply(lambda x: 1 if any(word in x.lower() for word in fraud_related_words) else 0)
email_df.head()

,body,to,from_,body_clean,fraud
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast,0
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...,0
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,test successful way go,0
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,randycan send schedule salary level everyone t...,0
5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com,greghow either next tuesday thursdayphillip,0


In [17]:
email_df.groupby('fraud').describe()

body                                                                 \
      count unique                                                top freq   
fraud                                                                        
0      9428   4717  george.huan@enron.com, mike.maggi@enron.com, l...   46   
1        36     21  jeffrey.hodge@enron.com, melissa.murphy@enron....    3   

         to                                                                 \
      count unique                                                top freq   
fraud                                                                        
0      9428   1585                            John Arnold/HOU/ECT@ECT  571   
1        36     16  aimee.shek@enron.com, albino.lopez@enron.com, ...    9   

      from_                                      body_clean         \
      count unique                     top  freq      count unique   
fraud                                                                
0      9428   1218   john.arnold@enron.com  2486       9428   4700   
1        36     14  outlook.team@enron.com     9         36     21   

                                                               
                                                     top freq  
fraud                                                          
0      georgehuanenroncom mikemaggienroncom larrymaye...   46  
1      jeffreyhodgeenroncom melissamurphyenroncomkyle...    3

In [18]:
observations_class_0 = email_df[email_df['fraud'] == 1]

In [19]:
observations_class_0

,body,to,from_,body_clean,fraud
1332,TheStreet.com and Privista are pleased to pres...,members@realmoney.com,members@realmoney.com,thestreetcom privista pleased present withcred...,1
3054,Premonition?=20=09=09=09British Trader Sentenc...,John Arnold/HOU/ECT@ECT,john.arnold@enron.com,premonition20090909british trader sentenced pr...,1
3326,The problem if we limit the size on options to...,andy.zipper@enron.com,john.arnold@enron.com,problem limit size option size offered onthe s...,1
3947,"paul.mead@enron.com, david.gallagher@enron.com...","richard.lewis@enron.com, john.lavorato@enron.c...",bob.shults@enron.com,paulmeadenroncom davidgallagherenroncomgregorb...,1
3949,Premonition?=20=09=09=09British Trader Sentenc...,John Arnold/HOU/ECT@ECT,john.arnold@enron.com,premonition20090909british trader sentenced pr...,1
4405,The problem if we limit the size on options to...,andy.zipper@enron.com,john.arnold@enron.com,problem limit size option size offered onthe s...,1
4505,"anitha.mathis@enron.com, antonette.concepcion@...","aimee.shek@enron.com, albino.lopez@enron.com, ...",outlook.team@enron.com,anithamathisenroncom antonetteconcepcionenronc...,1
4507,"anitha.mathis@enron.com, antonette.concepcion@...","aimee.shek@enron.com, albino.lopez@enron.com, ...",outlook.team@enron.com,anithamathisenroncom antonetteconcepcionenronc...,1
4508,"anitha.mathis@enron.com, antonette.concepcion@...","aimee.shek@enron.com, albino.lopez@enron.com, ...",outlook.team@enron.com,anithamathisenroncom antonetteconcepcionenronc...,1
5073,only you would know of a website that contradi...,=09Angie Conner (E-mail); Ann Sutton (E-mail);...,"=09Ward, Kim S (Houston) =20",would know website contradicts much time hand ...,1


## Taking care of missing values if present over here

In [20]:
email_df.isnull().sum()

body          0
to            0
from_         0
body_clean    0
fraud         0
dtype: int64

No missing values are there inside the dataset

In [21]:
X=email_df.drop('fraud',axis=1)
y=email_df['fraud']

In [22]:
X.head(2)

,body,to,from_,body_clean
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...


In [23]:
X.shape

(9464, 4)

In [24]:
y.head(2)

0    0
1    0
Name: fraud, dtype: int64

In [25]:
y.shape

(9464,)

## Cleaning the text

In [26]:
messages=X.copy()

In [27]:
messages['body'][1]

"Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time."

In [28]:
messages.reset_index(inplace=True)

In [29]:
messages

,index,body,to,from_,body_clean
0,0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com,forecast
1,1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com,traveling business meeting take fun trip espec...
2,2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com,test successful way go
3,3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com,randycan send schedule salary level everyone t...
4,5,"Greg,How about either next Tuesday or Thursday...",greg.piper@enron.com,phillip.allen@enron.com,greghow either next tuesday thursdayphillip
...,...,...,...,...,...
9459,9995,"shes pretty sexy, huh? are we getting togethe...",Brian Hoskins/HOU/ECT@ECT,eric.bass@enron.com,shes pretty sexy huh getting together talk eur...
9460,9996,i copied your idea - and it screwed up your name!,danielles@jonesgranger.com,eric.bass@enron.com,copied idea screwed name
9461,9997,"---------------------------Eric,Just a reminde...",Eric Bass/HOU/ECT@ECT,eric.bass@enron.com,ericjust reminder still need 9912 flash detail...
9462,9998,did you buy any enron in the 60s?,lwbthemarine@bigplanet.com,eric.bass@enron.com,buy enron 60


In [30]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['body_clean'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
print(corpus[1])

travel busi meet take fun trip especi prepar present would suggest hold busi plan meet take trip without formal busi meet would even tri get honest opinion whether trip even desir necessarya far busi meet think would product tri stimul discuss across differ group work often present speak other quiet wait turn meet might better held round tabl discuss formatmi suggest go austin play golf rent ski boat jet ski fli somewher take much time


In [32]:
voc_size=2500

In [33]:
from keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr[0]

[1614]

In [34]:
onehot_repr[1]

[2317,
 1742,
 1663,
 673,
 298,
 1471,
 1229,
 2281,
 164,
 1354,
 1035,
 634,
 1742,
 1991,
 1663,
 673,
 1471,
 1540,
 664,
 1742,
 1663,
 1354,
 1151,
 2058,
 1197,
 2288,
 661,
 1708,
 1471,
 1151,
 462,
 1094,
 1699,
 1742,
 1663,
 1704,
 1354,
 1569,
 2058,
 2188,
 2207,
 661,
 1003,
 1915,
 485,
 879,
 164,
 2320,
 2006,
 1902,
 1417,
 2220,
 1663,
 1507,
 123,
 577,
 2234,
 2209,
 2207,
 50,
 1035,
 1102,
 34,
 479,
 2270,
 872,
 1931,
 597,
 349,
 1931,
 949,
 2227,
 673,
 70,
 50]

## Embedding Representation

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...    0    0 1614]
 [ 577 2234 2209 ...  673   70   50]
 [   0    0    0 ... 2081 2390 1102]
 ...
 [   0    0    0 ...  233  213 2093]
 [   0    0    0 ...    0 1030  574]
 [2078  222  170 ...  163 1197 2232]]


In [36]:
embedded_docs[1]

array([ 577, 2234, 2209, 2207,   50, 1035, 1102,   34,  479, 2270,  872,
       1931,  597,  349, 1931,  949, 2227,  673,   70,   50], dtype=int32)

## Part 2 - Building and Training the RNN

In [37]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

## Initialising the RNN

In [38]:
regressor = Sequential()

## Adding Embedding layers over here

In [39]:
embedding_vector_features=40
regressor.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

## Adding the first LSTM layer and some Dropout regularisation

In [40]:
regressor.add(LSTM(units = 300, return_sequences = True))
regressor.add(Dropout(0.2))

## Adding the second LSTM layer and some Dropout regularisation

In [41]:
regressor.add(LSTM(units = 300, return_sequences = True))
regressor.add(Dropout(0.2))

## Adding the third LSTM layer and some Dropout regularisation

In [42]:
regressor.add(LSTM(units = 300, return_sequences = True))
regressor.add(Dropout(0.2))

## Adding the fourth LSTM layer and some Dropout regularisation

In [43]:
regressor.add(LSTM(units = 300, return_sequences = True))
regressor.add(Dropout(0.2))

## Adding the fifth LSTM layer and some Dropout regularisation

In [44]:
regressor.add(LSTM(units = 300))
regressor.add(Dropout(0.2))

## Adding the output layer over here

In [45]:
regressor.add(Dense(units = 1,activation="sigmoid"))

## Compiling the model

In [46]:
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])


In [47]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

## Splitting the dataset into training set and testing set over here

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.2,random_state=42,stratify=y_final)

## Dealing with imbalanced dataset over here





In [60]:
import imblearn
print(imblearn.__version__)

0.10.1


In [64]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Sample dataset - Replace this with your actual dataset
# Assume X contains the feature matrix and y contains the target labels




# Initialize SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE only to the training set
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Print the number of samples before and after oversampling
print("Before oversampling - Class 1 samples:", sum(y_train == 1))
print("After oversampling - Class 1 samples:", sum(y_train_resampled == 1))

# Now you can proceed with your training using the oversampled training data


Before oversampling - Class 1 samples: 29
After oversampling - Class 1 samples: 7542


In [65]:
X_train_resampled.shape

(15084, 20)

In [66]:
y_train_resampled.shape

(15084,)

## Training the model

In [73]:
regressor.fit(X_train_resampled, y_train_resampled, epochs = 50, batch_size = 32)

Epoch 1/50
472/472 [==============================] - 12s 25ms/step - loss: 0.6933 - accuracy: 0.5029
Epoch 2/50
472/472 [==============================] - 7s 15ms/step - loss: 0.6935 - accuracy: 0.4960
Epoch 3/50
472/472 [==============================] - 8s 17ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 4/50
472/472 [==============================] - 8s 17ms/step - loss: 0.6935 - accuracy: 0.4926
Epoch 5/50
472/472 [==============================] - 9s 19ms/step - loss: 0.6934 - accuracy: 0.4966
Epoch 6/50
472/472 [==============================] - 8s 17ms/step - loss: 0.6935 - accuracy: 0.4939
Epoch 7/50
472/472 [==============================] - 7s 15ms/step - loss: 0.6932 - accuracy: 0.5044
Epoch 8/50
472/472 [==============================] - 8s 18ms/step - loss: 0.6934 - accuracy: 0.5002
Epoch 9/50
472/472 [==============================] - 7s 15ms/step - loss: 0.6932 - accuracy: 0.5001
Epoch 10/50
472/472 [==============================] - 8s 17ms/step - loss: 0.6933 - accur

## Predicting the test set results

In [68]:
y_pred=regressor.predict(X_test)

60/60 [==============================] - 0s 7ms/step


## Setting the threshold value of 60%bold text

In [69]:
y_pred=np.where(y_pred > 0.6, 1,0)

## Evaluating the performance of the model

In [70]:
from sklearn.metrics import confusion_matrix

In [71]:
confusion_matrix(y_test,y_pred)

array([[1886,    0],
       [   7,    0]])

In [72]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9963021658742737

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1884
           1       0.00      0.00      0.00         9

    accuracy                           1.00      1893
   macro avg       0.50      0.50      0.50      1893
weighted avg       0.99      1.00      0.99      1893



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
